In [3]:
import os
import numpy as np
import scipy
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

In [5]:
# Load enwik 8

import re
file = open("/home/mike/Загрузки/nla/data/enwik8.txt", "r")
doclist = [line for line in file]
docstr = ''.join(doclist)
sentences = re.split(r'[.!?]', docstr)
sentences = [sentence.split() for sentence in sentences if len(sentence) > 1]

In [6]:


class word_to_vec():
    def __init__(self, text, min_frequancy):
        self.text = text
        self.k = min_frequancy
        self.vocabulary = {}
        
    def create_vocabulary(self, text, min_frequancy):
        vocabulary = {}
        draft_vocabulary = {}
        for sentence in text:
            for word in sentence:
                if word in draft_vocabulary:
                    draft_vocabulary[word] += 1
                else:
                    draft_vocabulary.update({word:1})
        value = 0
        for key in draft_vocabulary:
            if draft_vocabulary[key] > min_frequancy:
                vocabulary.update({key: value})
                value += 1
        del draft_vocabulary
        self.vocabulary = vocabulary
        
    #-----------------------------------------------
    
    def create_corpus_matrix(self, text, vocabulary):

        def if_in_vocabulary(word, context):
            if word in vocabulary and context in vocabulary:
                return True
            return False
    
        n = len(vocabulary)
        corpus_matrix = np.zeros((n, n))
    
        for sentense in text:
            for position_of_word in range(len(sentense)):
                window = range(max(0, position_of_word - 2), min(position_of_word + 3, len(sentense)))
                for i in window:
                    word = sentense[position_of_word]
                    context = sentense[i]
                    if if_in_vocabulary(word, context) and i != position_of_word:
                        corpus_matrix[vocabulary[word]][vocabulary[context]] += 1
                
        self.corpus_matrix = corpus_matrix
        
    #--------------------------------------------------
    
    def compute_embeddings(self, corpus_matrix, k, d=200):
        n = np.sum(corpus_matrix)
        ppmi = np.zeros(corpus_matrix.shape)
        sharp_w = corpus_matrix.sum(axis = 0)
        sharp_c = corpus_matrix.sum(axis = 1)
        for w in range(len(corpus_matrix)):
            for c in range(len(corpus_matrix[w])):
                if sharp_w[w] == 0 or sharp_c[c] == 0 or corpus_matrix[w][c] == 0:
                    ppmi[w][c] = 0
                else:
                    ppmi[w][c] = max(0, np.log((n * corpus_matrix[w][c])/(sharp_w[w]*sharp_c[c]*k)))
                
        sppmi = scipy.sparse.csc_matrix(ppmi)
        del ppmi
    
        u, sigma, v = svds(sppmi, d)
        embedding_matrix = np.matmul(u, np.diag(np.sqrt(sigma)))   
    
        self.embedding_matrix = embedding_matrix
    #---------------------------------------------------------
    
    def word_vector(self, word):
        word_vector = self.embedding_matrix[self.vocabulary[word]]
        return word_vector
    
    def nearest_words_for_word(self, word, top_n=10):
        word_embedding = self.word_vector(word)
        currword_num = self.vocabulary[word]
        
        similarities = cosine_similarity(self.embedding_matrix)
        all_words = {}
        for word_, wordnum_ in self.vocabulary.items():
            all_words.update({word_: similarities[wordnum_][currword_num]})
              
        sorted_words = sorted(all_words.items(), key=lambda x: x[1], reverse=True)
        neighbors = sorted_words[1:top_n+1]
        return neighbors
    
    def closest_word_for_vector(self, vector):
        embeddings = self.embedding_matrix
        maxsim = 0
        wordnum = 0
        word = None
        for embedding in embeddings:
            if 1 - scipy.spatial.distance.cosine(embedding, vector) > maxsim:
                maxsim = 1 - scipy.spatial.distance.cosine(embedding, vector)
                best_word_num = wordnum
                wordnum += 1
            else:
                wordnum += 1
        for k,v in ww.vocabulary.items():
            if v == best_word_num:
                word = k        
        return word, maxsim

In [7]:
ww = word_to_vec(sentences, 200)

In [8]:
ww.create_vocabulary(ww.text, ww.k)

In [9]:
ww.create_corpus_matrix(ww.text, ww.vocabulary)

In [10]:
ww.compute_embeddings(ww.corpus_matrix, k=5, d=200)

In [145]:
x = ww.word_vector('france') - ww.word_vector('paris') 
print(ww.closest_word_for_vector(x))
y = ww.word_vector('germany') - ww.word_vector('berlin') 
print(ww.closest_word_for_vector(y))
print(1 - scipy.spatial.distance.cosine(x,y))

('denmark', 0.44775455383549123)
('kingdom', 0.5917670748821964)
0.6431930687270788


In [61]:
ww.closest_word_for_vector(v)[0]

'germany'

In [140]:
len(ww.vocabulary)

5758

In [37]:
raw_test = open("/home/mike/Загрузки/nla/test.txt", "r")

In [38]:
raw_test_lines = [line.split('\n')[0] for line in raw_test]

In [39]:
raw_test_lines

[': capital-common-countries',
 'Athens Greece Baghdad Iraq',
 'Athens Greece Bangkok Thailand',
 'Athens Greece Beijing China',
 'Athens Greece Berlin Germany',
 'Athens Greece Bern Switzerland',
 'Athens Greece Cairo Egypt',
 'Athens Greece Canberra Australia',
 'Athens Greece Hanoi Vietnam',
 'Athens Greece Havana Cuba',
 'Athens Greece Helsinki Finland',
 'Athens Greece Islamabad Pakistan',
 'Athens Greece Kabul Afghanistan\t',
 'Athens Greece London England',
 'Athens Greece Madrid Spain\t',
 'Athens Greece Moscow Russia\t',
 'Athens Greece Oslo Norway\t',
 'Athens Greece Ottawa Canada\t',
 'Athens Greece Paris France\t',
 'Athens Greece Rome Italy\t',
 'Athens Greece Stockholm Sweden\t',
 'Athens Greece Tehran Iran\t',
 'Athens Greece Tokyo Japan\t',
 'Baghdad Iraq Bangkok Thailand',
 'Baghdad Iraq Beijing China',
 'Baghdad Iraq Berlin Germany',
 'Baghdad Iraq Bern Switzerland',
 'Baghdad Iraq Cairo Egypt',
 'Baghdad Iraq Canberra Australia',
 'Baghdad Iraq Hanoi Vietnam',
 'Bagh

In [45]:
rt = []
for line in raw_test_lines:
    if len(line)>0:
        if line[0] != ':':
            parsedline = []
            for word in line.split():
                parsedline.append(word.lower())
            rt.append(parsedline)

In [49]:
test_for_us = []

In [52]:
for line in rt:
    wordsin = 0
    for word in line:
        if word in ww.vocabulary:
            wordsin += 1
    if wordsin == len(line):
        test_for_us.append(line)
        

In [55]:
with open('our_test.txt', 'w') as f:
    for item in test_for_us:
        f.write("%s\n" % item)

In [137]:
for line in test_for_us:
    for word in line:
        if '.' in word:
            print(word)

In [119]:
for line in test_for_us:
    a, b, c, d = line
    if a == 'king':
        print(a,b,c,d)
        v = ww.word_vector(a) - ww.word_vector(c) + ww.word_vector(d)
        ans = ww.closest_word_for_vector(v)
        print(ans)
        
        print(np.linalg.norm(ww.word_vector(a)))
        print(np.linalg.norm(ww.word_vector(b)))
        print(np.linalg.norm(ww.word_vector(c)))
        print(np.linalg.norm(ww.word_vector(d)))
        print()
        print(np.linalg.norm(ww.word_vector(a) - ww.word_vector(b)))
        print(np.linalg.norm(v))
        break
    

king queen man woman
('king', 0.9090345717245961)
1.4238632858364706
1.3420759875497592
0.4494869209322624
0.7296107626736985

1.7252178002871625
1.0


In [148]:
a, b, c, d = test_for_us[0]
print(a,b,c,d)

#v = ww.word_vector(a) + ww.word_vector(b) + ww.word_vector(c)
#ans = ww.closest_word_for_vector(v)
    
print(vwkeras.most_similar(positive=[a], negative=[b])[0])

print(vwkeras.most_similar(positive=[c], negative=[d])[0])

athens greece beijing china
('stadium', 0.4736981689929962)
('recreation', 0.5182287693023682)


/home/mike/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/home/mike/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/home/mike/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [124]:
v = ww.word_vector('paris') - ww.word_vector('france') + ww.word_vector('germany')
print(ww.closest_word_for_vector(v))

('paris', 0.7515759114808241)


In [107]:
print(np.linalg.norm(ww.word_vector(a)))
print(np.linalg.norm(ww.word_vector(b)))
print(np.linalg.norm(ww.word_vector(c)))
print(np.linalg.norm(ww.word_vector(d)))
print()
print(np.linalg.norm(ww.word_vector(a) - ww.word_vector(b)))
print(np.linalg.norm(v))

1.1398531520241615
1.2089575376645667
1.4334160842491799
1.0647453565385052

1.4264212194831234
2.3451330422950085


In [56]:
import gensim
from gensim.models.word2vec import Word2Vec

In [57]:
vwkeras = Word2Vec(gensim.models.word2vec.LineSentence('/home/mike/Загрузки/nla/data/enwik8.txt'),iter=10)

In [64]:
i = 0
y = []
y_by_vw_svd = []
y_by_vw_keras = []
for line in test_for_us:
    a, b, c, d = line
    v = ww.word_vector(a) - ww.word_vector(b) + ww.word_vector(c)
    y_by_vw_svd.append(ww.closest_word_for_vector(v))
    y_by_vw_keras.append(vwkeras.most_similar(positive=[a,c], negative=[b])[0])
    y.append(d)
    i += 1
    if i % 10 == 0:
        print(i)   

/home/mike/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/home/mike/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [120]:
test_for_us[:10]

[['athens', 'greece', 'beijing', 'china'],
 ['athens', 'greece', 'berlin', 'germany'],
 ['athens', 'greece', 'london', 'england'],
 ['athens', 'greece', 'moscow', 'russia'],
 ['athens', 'greece', 'paris', 'france'],
 ['athens', 'greece', 'rome', 'italy'],
 ['athens', 'greece', 'tokyo', 'japan'],
 ['beijing', 'china', 'berlin', 'germany'],
 ['beijing', 'china', 'london', 'england'],
 ['beijing', 'china', 'moscow', 'russia']]

In [97]:
y_by_vw_keras[:10]

[('ithaca', 0.6578166484832764),
 ('tower', 0.6090428829193115),
 ('london.', 0.6776968836784363),
 ('kabul', 0.59325110912323),
 ('villa', 0.6682836413383484),
 ('abbey', 0.6600865721702576),
 ('birmingham', 0.6829279661178589),
 ('cologne', 0.6737047433853149),
 ('hall.', 0.6937240958213806),
 ('helsinki', 0.6732827425003052)]

In [68]:
svd_correct = 0
keras_correct = 0
for i in range(len(y)):
    if y[i] == y_by_vw_svd[i][0]:
        svd_correct += 1
    if y[i] == y_by_vw_keras[i][0]:
        keras_correct += 1
print(1.0 * svd_correct/len(y))
print(1.0 * keras_correct)
    

0.0
26.0
